<a href="https://colab.research.google.com/github/FauziRahmatullahSiregar/Significant-Weather/blob/main/df_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q metpy
!pip install -q ecmwflibs
!pip install -q eccodes
!pip uninstall -y -q xarray
!pip install -q xarray cfgrib

import xarray as xr
import numpy as np
import requests
import matplotlib.pyplot as plt
from metpy.units import units
import metpy.calc as mpcalc
import pickle
import requests

def calculate_tsi(dataset):
    '''
    Calculate the Thunderstorm Index (TSI) based on the TTI, KI, and the relative humidity
    :param dataset: xarray dataset with temperature and relative humidity
    :return: xarray dataset with the TSI
    '''

    # Create a grid of zeros
    grid = np.zeros((dataset.sizes['latitude'], dataset.sizes['longitude']))
    dataset = dataset.metpy.quantify()

    # Calculate Dewpoint
    dew = mpcalc.dewpoint_from_relative_humidity(dataset['t'], dataset['r']).metpy.dequantify()
    dew_850 = dew.sel(isobaricInhPa=850)
    dew_700 = dew.sel(isobaricInhPa=700)

    temp_850 = dataset['t'].sel(isobaricInhPa=850).metpy.convert_units('degC').metpy.dequantify()
    temp_700 = dataset['t'].sel(isobaricInhPa=700).metpy.convert_units('degC').metpy.dequantify()
    temp_500 = dataset['t'].sel(isobaricInhPa=500).metpy.convert_units('degC').metpy.dequantify()
    r_850 = dataset['r'].sel(isobaricInhPa=850).metpy.dequantify()
    r_700 = dataset['r'].sel(isobaricInhPa=700).metpy.dequantify()
    r_500 = dataset['r'].sel(isobaricInhPa=500).metpy.dequantify()

    TTI = temp_850 + dew_850 - (2 * temp_500)
    KI = (temp_850 - temp_500) + dew_850 - (temp_700 - dew_700)

    grid[(r_500 > 90) & (TTI > 44) & (KI > 25)] = 1

    tsi = xr.DataArray(grid, coords=[dataset['latitude'], dataset['longitude']], dims=['latitude', 'longitude'])
    return tsi


def calculate_ww(tp, vis, tcc, tsi):
    '''
    Calculate the Weather Category (WW) based on the TP, VIS, TCC, and TSI
    :param tp: xarray dataset with total precipitation
    :param vis: xarray dataset with visibility
    :param tcc: xarray dataset with total cloud cover
    :param tsi: xarray dataset with thunderstorm index
    :return: xarray dataset with the WW
    '''
    # Ensure all inputs are DataArrays
    tp = (tp.to_array() if isinstance(tp, xr.Dataset) else tp).squeeze()
    vis = (vis.to_array() if isinstance(vis, xr.Dataset) else vis).squeeze()
    tcc = (tcc.to_array() if isinstance(tcc, xr.Dataset) else tcc).squeeze()
    tsi = (tsi.to_array() if isinstance(tsi, xr.Dataset) else tsi).squeeze()

    # Create a grid of zeros
    grid = np.zeros((tp.sizes['latitude'], tp.sizes['longitude']))

    grid[(tsi == 1) & (tp <= 1)] = 17
    grid[(tsi == 1) & (tp > 1)] = 95
    grid[(tsi == 0) & (tp > 10)] = 65
    grid[(tsi == 0) & (tp > 5) & (tp <= 10)] = 63
    grid[(tsi == 0) & (tp > 1) & (tp <= 5)] = 61
    grid[(tsi == 0) & (tp <= 1) & (vis < 1000)] = 45
    grid[(tsi == 0) & (tp <= 1) & (vis <= 5000) & (vis >= 1000)] = 10
    grid[(tsi == 0) & (tp <= 1) & (vis > 5000) & (tcc < 10)] = 0
    grid[(tsi == 0) & (tp <= 1) & (vis > 5000) & (tcc < 60) & (tcc >= 10)] = 1
    grid[(tsi == 0) & (tp <= 1) & (vis > 5000) & (tcc < 90) & (tcc >= 60)] = 2
    grid[(tsi == 0) & (tp <= 1) & (vis > 5000) & (tcc > 90)] = 3

    ww = xr.DataArray(grid, coords=[tp['latitude'], tp['longitude']], dims=['latitude', 'longitude'])
    return ww

def load_data(model):
    # Load data for TSI
    ds_tsi = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa'}})[['t','r']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
    if model == 'GFS':
        # Load data for TP
        ds_tp1 = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'accum','typeOfLevel': 'surface','shortName':'tp'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tp']
        ds_tp2 = xr.load_dataset(filename_2, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'accum','typeOfLevel': 'surface','shortName':'tp'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tp']
        ds_tp = ds_tp2 - ds_tp1
        # Load data for Vis
        ds_vis = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'instant','typeOfLevel': 'surface','shortName':'vis'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['vis']

        #Total Cloud Cover
        ds_tcc = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'instant','typeOfLevel': 'atmosphere', 'shortName':'tcc'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tcc']

        # Load additional data
        ds_surface = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround','level':2.0}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
        ds_wind = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround','level':10.0}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
        r2 = ds_surface['r2']

    elif model == 'IFS':
        # Load data for TP
        ds_tp1 = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface','shortName':'tp'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tp']
        ds_tp2 = xr.load_dataset(filename_2, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface','shortName':'tp'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tp']
        ds_tp = ds_tp2 - ds_tp1

        # Load data for Vis
        ds_vis = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'instant','typeOfLevel': 'surface','shortName':'vis'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['vis']

        #Total Cloud Cover
        ds_tcc = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'instant','typeOfLevel': 'surface','shortName':'tcc'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tcc']

        # Load additional data
        ds_surface = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface','edition':1}})[['t2m','d2m']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
        r2 = mpcalc.relative_humidity_from_dewpoint(ds_surface['t2m'], ds_surface['d2m']).metpy.dequantify()
        ds_wind = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface','edition':1}})[['u10','v10']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')

    else:
        # Load data for TP
        ds_tp1 = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'accum','typeOfLevel': 'surface'}})[['tirf']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tirf']
        ds_tp2 = xr.load_dataset(filename_2, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'accum','typeOfLevel': 'surface'}})[['tirf']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['tirf']
        ds_tp = ds_tp2 - ds_tp1

        # Load data for Vis
        ds_vis = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'stepType': 'min','typeOfLevel': 'surface'}})[['unknown']].sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['unknown']

        #Total Cloud Cover
        ds_tcc = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface', 'stepType': 'instant', 'shortName':'unknown'}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')['unknown']

        # Load additional data
        ds_surface = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround','level':2.0}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
        ds_wind = xr.load_dataset(filename_1, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround','level':10.0}}).sel(latitude=lats, longitude=lons).interp(latitude=df_lats, longitude=df_lons, method='linear')
        r2 = ds_surface['r2']


    tsi = calculate_tsi(ds_tsi)
    ww = calculate_ww(ds_tp, ds_vis, ds_tcc, tsi)
    coords = ds_tp.coords

    ds_fc = xr.Dataset(
          {
          'tp': (['latitude','longitude'],ds_tp.values),
          'tsi': (['latitude','longitude'],tsi.values),
          't2m': (['latitude','longitude'], ds_surface['t2m'].values),
          'tcc': (['latitude','longitude'], ds_tcc.values),
          'vis': (['latitude','longitude'], ds_vis.values),
          'r2': (['latitude','longitude'], r2.values),
          },
          coords=coords)

    ds_fc_wind = xr.Dataset(
          {
          'u10': (['latitude','longitude'],ds_wind['u10'].values),

          'v10': (['latitude','longitude'],ds_wind['v10'].values),
          },
          coords=coords)

    ds_af = xr.Dataset(
          {
          'ww': (['latitude','longitude'],ww.values),
          },
          coords=coords)

    # Save the dataset
    ds_complete = xr.merge([ds_fc, ds_fc_wind, ds_af],compat='override')
    ds_complete.attrs = {
        'GRIB_edition': 2,
        'GRIB_centre': 'wiix',
        'GRIB_centreDescription': 'Indonesia Meteorological Climatological and Geophysical Agency - BMKG',
        'GRIB_subCentre': 0,
        'Conventions': 'CF-1.7',
        'institution': 'Indonesia Meteorological Climatological and Geophysical Agency - BMKG',
      }

    ds_complete.to_netcdf(f'df_auto_{model}_test.nc')
    return ds_complete

In [7]:
# Load the data

#If the data already exists
#filename_1 = '/content/drive/MyDrive/0001/A1D07251200072512011_R20240725120000_20240725120000_20240725120000.grib'
#filename_2 = '/content/drive/MyDrive/0001/A1D07251200072512011_R20240725120000_20240725120000_20240725120000.grib'

#'''If the data is downloaded first

!wget https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/00/atmos/gfs.t00z.pgrb2.0p25.f003
filename_1 = 'gfs.t00z.pgrb2.0p25.f003'

!wget https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/00/atmos/gfs.t00z.pgrb2.0p25.f006
filename_2 = 'gfs.t00z.pgrb2.0p25.f006'

#'''
# Domain slice
lats = slice (9, -13)
lons = slice (90, 143)

# Load lats lons from the harmonization
with open('/content/drive/MyDrive/0001/df_latitudes.pkl', 'rb') as f:
    df_lats = pickle.load(f)

with open('/content/drive/MyDrive/0001/df_longitudes.pkl', 'rb') as f:
    df_lons = pickle.load(f)

ds_complete = load_data('GFS')

--2024-08-02 01:46:31--  https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/00/atmos/gfs.t00z.pgrb2.0p25.f003
Resolving nomads.ncep.noaa.gov (nomads.ncep.noaa.gov)... 23.210.215.16, 23.210.215.43, 2600:1417:76::17d2:d710, ...
Connecting to nomads.ncep.noaa.gov (nomads.ncep.noaa.gov)|23.210.215.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 538373465 (513M)
Saving to: ‘gfs.t00z.pgrb2.0p25.f003.3’

gfs.t00z.pgrb2.0p25 100%[===================>] 513.43M   122MB/s    in 3.7s    

2024-08-02 01:46:34 (138 MB/s) - ‘gfs.t00z.pgrb2.0p25.f003.3’ saved [538373465/538373465]

--2024-08-02 01:46:35--  https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/00/atmos/gfs.t00z.pgrb2.0p25.f006
Resolving nomads.ncep.noaa.gov (nomads.ncep.noaa.gov)... 23.210.215.16, 23.210.215.43, 2600:1417:76::17d2:d710, ...
Connecting to nomads.ncep.noaa.gov (nomads.ncep.noaa.gov)|23.210.215.16|:443... connected.
HTTP request sent, awaiting response... 200 O

ERROR:cfgrib.dataset:skipping variable: paramId==228164 shortName='tcc'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cfgrib/dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.10/dist-packages/cfgrib/dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,  250.,
        200.,  150.,  100.,   70.,   50.,   40.,   30.,   20.,   15.,
         10.,    7.,    5.,    3.,    2.,    1.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,  250.,
        200.,  150.,  100.,